In [1]:
import pandas as pd
from scipy import spatial

In [36]:
df_substations = pd.read_excel('./substations.xlsx', index_col=0)

substation_locs = []
for index, row in df_substations.iterrows():
    substation_locs.append((row.lon, row.lat))
    
substation_tree = spatial.KDTree(substation_locs)

In [19]:
df_powerplants = pd.read_csv('./powerplants.csv')

relevant_countries = ['Germany', 'Luxembourg', 'Austria', 'Czech Republic', 'Slovenia', 'Belgium', 'Croatia', 'Hungary', 'Poland', 'France', 'Slovakia', 'Romania', 'Netherlands']
country_zone_map = {
    'Germany': 'DE', 
    'Luxembourg': 'DE', 
    'Austria': 'AT', 
    'Czech Republic': 'CZ', 
    'Slovenia': 'SI', 
    'Belgium': 'BE',
    'Croatia': 'HR', 
    'Hungary': 'HU', 
    'Poland': 'PL', 
    'France': 'FR', 
    'Slovakia': 'SK', 
    'Romania': 'RO', 
    'Netherlands': 'NL',
}

df_powerplants = df_powerplants[df_powerplants.Country.isin(relevant_countries)]
df_powerplants['zone'] = df_powerplants.apply(lambda row : country_zone_map[row.Country], axis=1)

display(df_powerplants[(df_powerplants.zone == 'FR') & (df_powerplants.Fueltype == 'Natural Gas')])
print(df_powerplants[(df_powerplants.zone == 'FR') & (df_powerplants.Fueltype == 'Natural Gas')].Capacity.sum())

df_powerplants.Technology.unique()


,id,Name,Fueltype,Technology,Set,Country,Capacity,Efficiency,DateIn,DateRetrofit,DateOut,lat,lon,Duration,Volume_Mm3,DamHeight_m,StorageCapacity_MWh,EIC,projectID,zone
129,129,Blenod,Natural Gas,CCGT,PP,France,427.0,NaN,2011.0,NaN,NaN,48.869590,6.083770,NaN,0.0,0.0,0.0,{'17W100P100P0243J'},"{'ENTSOE': {'17W100P100P0243J'}, 'OPSD': {'OEU...",FR
130,130,Bouchain,Natural Gas,CCGT,PP,France,585.0,0.610000,2017.0,2017.0,NaN,50.297740,3.314290,NaN,0.0,0.0,0.0,{'17W100P100P0244H'},"{'ENTSOE': {'17W100P100P0244H'}, 'OPSD': {'OEU...",FR
131,131,Combigolfe,Natural Gas,CCGT,PP,France,425.0,NaN,2010.0,2010.0,NaN,43.416160,4.869740,NaN,0.0,0.0,0.0,{'17W100P100P0337A'},"{'ENTSOE': {'17W100P100P0337A'}, 'OPSD': {'OEU...",FR
132,132,Croix De Metz,Natural Gas,CCGT,PP,France,413.0,0.570000,2019.0,2019.0,NaN,48.693360,5.914680,NaN,0.0,0.0,0.0,{'17W100P100P0340L'},"{'ENTSOE': {'17W100P100P0340L'}, 'OPSD': {'OEU...",FR
133,133,Dk,Natural Gas,CCGT,CHP,France,796.0,NaN,2005.0,2005.0,NaN,51.046280,2.318350,NaN,0.0,0.0,0.0,{'17W100P100P0350I'},"{'ENTSOE': {'17W100P100P0350I'}, 'OPSD': {'OEU...",FR
134,134,Martigues Ponteau,Natural Gas,CCGT,PP,France,930.0,0.565000,2012.0,NaN,NaN,43.358220,5.021120,NaN,0.0,0.0,0.0,"{'17W100P100P0246D', '17W100P100P0245F'}","{'ENTSOE': {'17W100P100P0246D', '17W100P100P02...",FR
135,135,Pont Sur Sambre,Natural Gas,CCGT,PP,France,412.0,NaN,2019.0,2019.0,NaN,50.229150,3.868070,NaN,0.0,0.0,0.0,{'17W100P100P0341J'},"{'ENTSOE': {'17W100P100P0341J'}, 'OPSD': {'OEU...",FR
302,302,Gennevilliers,Natural Gas,CCGT,PP,France,203.0,NaN,1994.0,NaN,NaN,48.936990,2.261440,NaN,0.0,0.0,0.0,{'17W100P100P02667'},"{'ENTSOE': {'17W100P100P02667'}, 'OPSD': {'OEU...",FR
416,416,Spem,Natural Gas,CCGT,PP,France,435.0,0.382118,2010.0,2010.0,NaN,47.302650,-2.140280,NaN,0.0,0.0,0.0,{'17W100P100P0334G'},"{'ENTSOE': {'17W100P100P0334G'}, 'OPSD': {'OEU...",FR
434,434,Cycofos,Natural Gas,CCGT,PP,France,428.0,NaN,2009.0,2009.0,NaN,43.428010,4.886520,NaN,0.0,0.0,0.0,{'17W100P100P0351G'},"{'ENTSOE': {'17W100P100P0351G'}, 'OPSD': {'OEU...",FR


7476.0


array(['Steam Turbine', 'CCGT', 'Pumped Storage', 'Run-Of-River',
       'Reservoir', nan, 'Marine', 'Onshore', 'Offshore', 'not found',
       'Combustion Engine'], dtype=object)

In [45]:
df_powerplants = pd.read_csv('./powerplants.csv')

relevant_countries = ['Germany', 'Luxembourg', 'Austria', 'Czech Republic', 'Slovenia', 'Belgium', 'Croatia', 'Hungary', 'Poland', 'France', 'Slovakia', 'Romania', 'Netherlands']
country_zone_map = {
    'Germany': 'DE', 
    'Luxembourg': 'DE', 
    'Austria': 'AT', 
    'Czech Republic': 'CZ', 
    'Slovenia': 'SI', 
    'Belgium': 'BE',
    'Croatia': 'HR', 
    'Hungary': 'HU', 
    'Poland': 'PL', 
    'France': 'FR', 
    'Slovakia': 'SK', 
    'Romania': 'RO', 
    'Netherlands': 'NL',
}

df_powerplants = df_powerplants[df_powerplants.Country.isin(relevant_countries)]
df_powerplants['zone'] = df_powerplants.apply(lambda row : country_zone_map[row.Country], axis=1)
df_powerplants = df_powerplants.drop(columns=['id', 'Duration', 'Volume_Mm3', 'StorageCapacity_MWh', 'DamHeight_m', 'DateIn', 'DateRetrofit', 'Country'])

df_powerplants = df_powerplants.rename(columns={
    'Name': 'name', 
    'Fueltype': 'fuel_type',
    'Technology': 'technology',
    'Set': 'set',
    'Capacity': 'capacity',
    'Efficiency': 'efficiency',
    'DateOut': 'date_out',
    'EIC': 'eic',
    'projectID': 'project_id',
})
df_powerplants = df_powerplants.reset_index(drop=True)

In [48]:
for index, row in df_powerplants.iterrows():
    result = substation_tree.query([(row.lon,row.lat)])
    df_powerplants.loc[index, 'node'] = result[1][0]

In [49]:
display(df_powerplants)

,name,fuel_type,technology,set,capacity,efficiency,date_out,lat,lon,eic,project_id,zone,node
0,Emsland,Nuclear,Steam Turbine,CHP,1336.000000,0.33,2022.0,52.481878,7.306658,{'11WD7KKE-1K--KW5'},"{'ENTSOE': {'11WD7KKE-1K--KW5'}, 'OPSD': {'BNA...",DE,343
1,Mellach,Hard Coal,Steam Turbine,CHP,200.000000,NaN,2020.0,46.911700,15.488300,{'14W-WML-KW-----0'},"{'BEYONDCOAL': {'BEYOND-AT-11'}, 'ENTSOE': {'1...",AT,1091
2,Eemshaven,Hard Coal,CCGT,PP,1604.170304,0.58,2029.0,53.440500,6.861200,{'49W000000000044-'},"{'BEYONDCOAL': {'BEYOND-NL-12'}, 'ENTSOE': {'4...",NL,818
3,Emile Huchet,Hard Coal,CCGT,PP,596.493211,NaN,2022.0,49.152500,6.698100,"{'17W100P100P0345B', '17W100P100P0344D'}","{'BEYONDCOAL': {'BEYOND-FR-67'}, 'ENTSOE': {'1...",FR,446
4,Belleville,Nuclear,Steam Turbine,PP,2620.000000,NaN,2028.0,47.508790,2.875740,"{'17W100P100P01865', '17W100P100P01857'}","{'ENTSOE': {'17W100P100P01865', '17W100P100P01...",FR,1085
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2160,Sandbank,Wind,Offshore,PP,288.000000,NaN,NaN,51.165691,10.451526,{'11WD2OWS0000352F'},{'ENTSOE': {'11WD2OWS0000352F'}},DE,1472
2161,Staudinger,Natural Gas,CCGT,PP,1132.000000,NaN,NaN,51.165691,10.451526,{'11WD2STAU0001800'},{'ENTSOE': {'11WD2STAU0001800'}},DE,1472
2162,Westermeerwind,Wind,Offshore,PP,144.000000,NaN,NaN,52.132633,5.291266,{'49W0000000001411'},{'ENTSOE': {'49W0000000001411'}},NL,76
2163,Wikinger Prod,Wind,Offshore,PP,350.000000,NaN,NaN,51.165691,10.451526,{'11W0-0000-0026-Y'},{'ENTSOE': {'11W0-0000-0026-Y'}},DE,1472


In [50]:
df_powerplants.to_excel("power_plants.xlsx")